In [1]:
import numpy as np
import vedo

from histalign.backend.ccf.paths import get_atlas_path
from histalign.backend.io import load_volume
from histalign.backend.maths import compute_normal
from histalign.backend.models import (
    AlignmentSettings,
    HistologySettings,
    Orientation,
    Resolution,
    VolumeSettings,
)
from histalign.backend.workspace import VolumeSlicer
from notebook_helpers import show, update_axes, update_cameras, coronal_camera

In [2]:
def euclidean_distance(point1: np.ndarray, point2: np.ndarray) -> float:
    return np.sqrt(np.sum((point1 - point2) ** 2))


def find_nearest_grid_point(point: np.ndarray) -> np.ndarray:
    return np.round(point)


def compute_stack_points(starting_point, normal, z_size, z_spacing, snap: bool = False):
    middle_index = (z_size - 1) // 2

    points = []
    for i in range(z_size):
        distance = (i - middle_index) * z_spacing
        point = starting_point + distance * normal
        if snap:
            point = find_nearest_grid_point(free_point)
        points.append(point)

    return points

In [3]:
orientation = Orientation.CORONAL
resolution = Resolution.MICRONS_25

atlas_volume = load_volume(get_atlas_path(resolution))
update_cameras(atlas_volume.tonumpy().shape)
update_axes(atlas_volume.tonumpy().shape)

In [4]:
shape = atlas_volume.tonumpy().shape
z_stack_count = 21
z_stack_distance = 10

alignment_coordinates = np.array(shape) // 2
pitch = 10
yaw = 4

settings = AlignmentSettings(
    volume_path=get_atlas_path(resolution),
    volume_settings=VolumeSettings(
        orientation=orientation,
        resolution=resolution,
        pitch=pitch,
        yaw=yaw,
    ),
    histology_settings=HistologySettings(),
)

In [5]:
alignment_normal = compute_normal(settings.volume_settings)
alignment_plane = VolumeSlicer(volume=atlas_volume).slice(
    settings.volume_settings, return_mesh=True
)

normal_line_point1 = alignment_coordinates - 1000 * alignment_normal
normal_line_point2 = alignment_coordinates + 1000 * alignment_normal

In [10]:
point_coordinates = compute_stack_points(
    alignment_coordinates, alignment_normal, z_stack_count, z_stack_distance
)
points = [vedo.Point(point) for point in point_coordinates]
show(
    [
        alignment_plane,
        # points,
        *[
            VolumeSlicer(volume=atlas_volume).slice(
                settings.volume_settings, origin=point, return_mesh=True
            )
            for point in point_coordinates
        ],
    ],
    camera=coronal_camera,
)